In [1]:
counterstainAssignment = False
controlImage = True
counterstainControlPath = None
smFISHChannelPath = "/Users/eliasguan/Desktop/306_analysis_results/Experiment/6hr_Amputation/Image1/565/6hr_Amputation_Image1_565.tif"
counterstainChannelPath = None
nucleiSegmentationPath = "/Users/eliasguan/Desktop/306_analysis_results/Experiment/6hr_Incision/Image1/565/results/labels"

In [2]:
plotSpotLabel = False 
labelExpansionSize = 20
nuclei_projection_size = 7

In [3]:
# This part is for importing all the functions for smFISH detection. Please install them if you dont have these pacakges. 
import os
import sys
# import tk for getting the directory faster. dont need this in a command line/server version
import tkinter as tk
from tkinter import *
from tkinter import filedialog
import numpy as np
import bigfish.detection 
import bigfish.stack
import bigfish.plot
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import random
import math
import json
# if you dont need to plot in jupyter you don need these. Some magic interperters need to be removed for command line version. 
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Glob and tifffile are needed
from glob import glob
from tifffile import imread,imwrite
# csb deep is to take normalization 
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible
# This is your stardist models and everything in stardist coming from. 
from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D
from skimage import segmentation
import bigfish.multistack as multistack
# Set random seed for you color map. You do not really need this to be 6 all the time, but its okay. 
np.random.seed(6)
lbl_cmap = random_label_cmap()

In [4]:
def load_npy_file(filename):
    try:
        data = np.load(filename)
        print(f"Loaded {filename} successfully.")
        return data
    except FileNotFoundError:
        print(f"{filename} not found.")
    except Exception as e:
        print(f"Error loading {filename}: {e}")
    return None
def make_stardist_Predictions_labels (dataset, normalized = False, normalize_low = 0, normalize_high = 0, nms_thresh = 0, prob_thresh = 0): 
    ''' input: csbdeep input: If this image is a csbdeep filtered image. If this is true, the image will not be normalized. 
               dataset: The data set you want to analysis on 
               normalize_low and normalized_high: The parameter you want to normalize, if either of them is 0 will use default value (1,99.8)
               if not then will use these values 
               nms_thresh and prob_thrsh : the parameter of overlapping and probablity threshold. If eitehr of them is 0 will use default value 
               for the model (depend on the model) if not then use these values
    '''
    labels_collection= []
    for i in range(len(dataset)): 
        if not normalized:
            if normalize_low ==0 or normalize_high == 0:
                img = normalize(dataset[i], 1,99.8, axis=(0,1))
            else:
                img = normalize(dataset[i], normalize_low, normalize_high, axis=(0,1))
        else:
            img = dataset[i]
        if nms_thresh == 0 or prob_thresh == 0 :
            labels, details = model.predict_instances(img)
        else: 
            labels, details = model.predict_instances(img,nms_thresh = nms_thresh, prob_thresh = prob_thresh)
        # write labels
        labels_collection.append(labels)
        export_imagej_rois('polygons/polygon_rois_'+str(i).zfill(3)+'.zip', details['coord'])
        imwrite("labels/Nucleus_Labels_"+str(i).zfill(3)+".tif", labels)
    labels_collection = np.array(labels_collection, dtype = np.uint16)
    return labels_collection
def random_select_images(dataset, percentage):
    """
    Randomly select a certain percentage of images from the dataset.

    Parameters:
        dataset (list): A list containing 2D arrays (images).
        percentage (float): Percentage of images to be selected.

    Returns:
        selected_indices (list): A list of indices corresponding to the selected images.
    """
    num_images = len(dataset)
    num_selected = int(np.ceil(percentage/100 * num_images))
    selected_indices = np.random.choice(num_images, num_selected, replace=False)
    return selected_indices

def random_select_region(image, region_size):
    """
    Randomly select a region within the image with a certain size.

    Parameters:
        image (2D array): The original image.
        region_size (tuple): The size of the region to be selected (height, width).

    Returns:
        region (2D array): The selected region.
    """
    image_height, image_width = image.shape
    region_height, region_width = region_size

    if region_height > image_height or region_width > image_width:
        raise ValueError("Region size exceeds original image size.")

    start_row = np.random.randint(0, image_height - region_height + 1)
    start_col = np.random.randint(0, image_width - region_width + 1)
    end_row = start_row + region_height
    end_col = start_col + region_width

    region_prop = [(start_row, end_row),(start_col,end_col)]
    return region_prop
def make_random_examples(dataset, normalize_low = 0, normalize_high =0, nms_thresh =0, prob_thresh = 0, normalized = False, percentage = 20, region_size = (500,500)):
    selected_indicies = sorted(random_select_images(dataset, percentage))
    for item in selected_indicies: 
        if normalized:
            img = dataset[item]
        else:
            if normalize_low ==0 or normalize_high == 0:
                img = normalize(dataset[item], 1,99.8, axis=(0,1))
            else:
                img = normalize(dataset[item], normalize_low, normalize_high, axis=(0,1))   
        if nms_thresh == 0 or prob_thresh == 0 :
            labels, details = model.predict_instances(img)
        else: 
            labels, details = model.predict_instances(img, nms_thresh = nms_thresh, prob_thresh = prob_thresh)
        region_props =  random_select_region(img, region_size)
        cropped_image = img[region_props[0][0]:region_props[0][1],region_props[1][0]:region_props[1][1]]
        cropped_label = labels[region_props[0][0]:region_props[0][1],region_props[1][0]:region_props[1][1]]
        figure = plt.figure(figsize=(13,10))
        coord, points, prob = details['coord'], details['points'], details['prob']
        # Plot image on the first one
        ax1 = figure.add_subplot(121); ax1.imshow(cropped_image, cmap='gray'); ax1.axis('off')
        # Plot image on the second one
        ax2 = figure.add_subplot(122); ax2.imshow(cropped_image, cmap='gray'); ax2.axis('off')
        # Plot labels on the third one. 
        ax2.imshow(cropped_label, cmap=lbl_cmap, alpha=0.3)
        figure.tight_layout()
        plt.savefig("random_examples/random_example_"+str(item).zfill(3)+".tif")
def create_folder_in_same_directory(file_path, folder_name):
    """
    Creates a folder with the specified name in the same directory as the given file.
    If the folder already exists, it returns the existing path.
    """
    # Get the directory of the given file
    directory = os.path.dirname(file_path)
    
    # Define the path for the specified folder
    folder_path = os.path.join(directory, folder_name)
    
    # Check if the folder exists
    if not os.path.exists(folder_path):
        # Create the folder if it doesn't exist
        os.makedirs(folder_path)
        print(f"Created '{folder_name}' folder at: {folder_path}")
    else:
        print(f"'{folder_name}' folder already exists at: {folder_path}")
    
    return folder_path
def generate_max_projection_array(array, projection_size):
    ranges = []
    total = array.shape[0]
    projected_image = []
    for i in range(0, total, projection_size):
        start = i
        end = min(i + projection_size - 1, total - 1)
        ranges.append((start, end))
    for item in ranges:
        start, end = item

        for i in range(start,end):
            nuclei_array.append(array[i])
        projection = bigfish.stack.maximum_projection(np.array(nuclei_array,dtype=np.uint8))
        projected_image.append(projection)
    return np.array(projected_image,dtype=np.uint8)

In [5]:
# Get into your counterstain spots
if counterstainAssignment:
    os.chdir(os.path.dirname(counterstainChannelPath))
    os.chdir("results")
    # Read in your counterstain spots file 
    # File names
    file_A = 'spots_post_decomposition_and_background_removed.npy'
    file_B = 'spots_post_decomposition.npy'

    # Try loading A, fallback to B if A fails
    post_decomposition_array = load_npy_file(file_A)
    if post_decomposition_array is None:
        post_decomposition_array = load_npy_file(file_B)
    post_decomposition_array_projected = np.copy(post_decomposition_array)  # Create a copy of the original array
    post_decomposition_array_projected[:, 0] = np.floor_divide(post_decomposition_array[:, 0], nuclei_projection_size)

In [6]:
os.chdir(nucleiSegmentationPath)
nucleiFileNames = sorted(glob("*.tif"))
nucleiImageArray_projected_labels = []
for item in nucleiFileNames:
    nucleiImage = imread(item)
    nucleiImageArray_projected_labels.append(nucleiImage)

In [7]:
if counterstainAssignment:
    os.chdir(os.path.dirname(counterstainChannelPath))
    os.chdir("results")
    create_folder_in_same_directory(".", "expanded_labels")
    create_folder_in_same_directory(".", "counterstain_labels")
    create_folder_in_same_directory(".", "counterstain_assignment_maps")
    spot_in_background = []
    counterstain_assignment_results = []
else:
    os.chdir(os.path.dirname(smFISHChannelPath))
    os.chdir("results")
    create_folder_in_same_directory(".", "expanded_labels")
    create_folder_in_same_directory(".", "assignment_maps")
for i in range(len(nucleiImageArray_projected_labels)):
    # Expanding labels 
    expanded_labels = segmentation.expand_labels(nucleiImageArray_projected_labels[i], distance=labelExpansionSize)
    # Find your spots belong to this projection 
    if counterstainAssignment:
        indices = np.where(post_decomposition_array_projected[:, 0] == i)[0]
        # Remove the z-stack information from your RNA detection
        rna_coord = post_decomposition_array_projected[indices][:, -2:]
        # Data type configureation. Changing dtype will probablly 
        # Get you more compatible with bigfish also makes calculation easier
        rna_coord = rna_coord.astype(dtype="int64")
        cell_RNACount = np.zeros(np.max(expanded_labels)+1)
    expanded_labels = expanded_labels.astype(dtype=np.uint16)
    
    # Save your expanded labels 
    imwrite('expanded_labels/20expandedlabel'+str(i).zfill(3)+'.tif', expanded_labels, photometric = 'minisblack')
    # Get cell RNA counter as a zero array 
    if counterstainAssignment:
        # Iterate through your RNA coordinates 
        for item in rna_coord:
            # Find the value of this RNA center spot
            location = expanded_labels[item[0],item[1]]
            # Add on to your RNA counter
            cell_RNACount[location] = cell_RNACount[location]+1
        # Add your results to your collection
        counterstain_assignment_results.append(cell_RNACount[1:])
        # Create empty assignment maps 
        assignment_map = np.zeros(expanded_labels.shape, dtype = np.uint8)
        assignment_label = np.zeros(expanded_labels.shape, dtype = np.uint64)
        # Create your maps 
        k = 1
        for j in tqdm(range(1,len(cell_RNACount))):
            indices = np.where(expanded_labels == j)
            assignment_map[indices] = cell_RNACount[j]
            if cell_RNACount[j] != 0 :
                assignment_label[indices] = k
                k = k+1
            else:
                assignment_label[indices] = 0
        spot_in_background.append(cell_RNACount[0])
        imwrite('counterstain_labels/assignment_map'+str(i).zfill(3)+'.tif', assignment_label, photometric = 'minisblack')
        imwrite('counterstain_assignment_maps/assignment_map'+str(i).zfill(3)+'.tif', assignment_map, photometric = 'minisblack')

Created 'expanded_labels' folder at: expanded_labels
Created 'assignment_maps' folder at: assignment_maps


In [8]:
if counterstainAssignment:
    np.savez("counterstain_assignment_result.npz", *counterstain_assignment_results)
    np.save("counterstain_spot_in_background.npy", spot_in_background)
    np.save("concatenated_spot_counterstain.npy", np.concatenate(counterstain_assignment_results))